# Логистическая регрессия

Обучим логистическую регрессию для предсказания того, откликнется клиент  
на рекламное предложение (target = 1) или нет (target = 0).

## Подлючение библиотек, загрузка и обзор данных

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

In [2]:
data = pd.read_csv('https://raw.githubusercontent.com/evgpat/edu_stepik_practical_ml/main/datasets/clients_data.csv')

In [3]:
data.head()

,AGE,SOCSTATUS_WORK_FL,SOCSTATUS_PENS_FL,GENDER,CHILD_TOTAL,DEPENDANTS,PERSONAL_INCOME,LOAN_NUM_TOTAL,LOAN_NUM_CLOSED,LOAN_DLQ_NUM,TARGET
0,49,1,0,1,2,1,5000.0,1,1,2,0
1,32,1,0,1,3,3,12000.0,1,1,1,0
2,52,1,0,1,4,0,9000.0,2,1,0,0
3,39,1,0,1,1,1,25000.0,1,1,3,0
4,30,1,0,0,0,0,12000.0,2,1,2,0


In [4]:
from sklearn.model_selection import train_test_split

X = data.drop('TARGET', axis=1)
y = data['TARGET']

Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, train_size=0.7, random_state=123)

## Практика

**Задание**  
Выведите на экран количество объектов каждого класса. Сколько процентов объектов относятся к положительному классу?  
Ответ округлите до целого числа (например, если доля объектов положительного класса равна 0.412, в ответ запишите 41,  
имея в виду 41 процент).

In [5]:
data.describe()

,AGE,SOCSTATUS_WORK_FL,SOCSTATUS_PENS_FL,GENDER,CHILD_TOTAL,DEPENDANTS,PERSONAL_INCOME,LOAN_NUM_TOTAL,LOAN_NUM_CLOSED,LOAN_DLQ_NUM,TARGET
count,15223.000000,15223.000000,15223.000000,15223.000000,15223.000000,15223.000000,15223.000000,15223.000000,15223.000000,15223.000000,15223.000000
mean,40.406096,0.909610,0.134468,0.654536,1.099389,0.645208,13853.836323,1.387769,0.751889,0.195362,0.119030
std,11.601068,0.286748,0.341165,0.475535,0.995411,0.812252,9015.467617,0.793566,0.988499,0.685022,0.323835
min,21.000000,0.000000,0.000000,0.000000,0.000000,0.000000,24.000000,1.000000,0.000000,0.000000,0.000000
25%,30.000000,1.000000,0.000000,0.000000,0.000000,0.000000,8000.000000,1.000000,0.000000,0.000000,0.000000
50%,39.000000,1.000000,0.000000,1.000000,1.000000,0.000000,12000.000000,1.000000,0.000000,0.000000,0.000000
75%,50.000000,1.000000,0.000000,1.000000,2.000000,1.000000,17000.000000,2.000000,1.000000,0.000000,0.000000
max,67.000000,1.000000,1.000000,1.000000,10.000000,7.000000,250000.000000,11.000000,11.000000,13.000000,1.000000


Обучим логистическую регрессию с параметрами по умолчанию.

In [17]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()

In [18]:
lr.fit(Xtrain, ytrain)

prediction = lr.predict(Xtest)

Метрику accuracy не стоит использовать при сильном дисбалансе классов. Поэтому посчитайте f1_score для оценки качества модели на тестовых данных.

f1_score принимает значения от 0 до 1. Чем ближе к 1, тем лучше модель.

In [19]:
from sklearn.metrics import f1_score

f1_score(ytest, prediction)

0.0

**Вопрос**  
Чему равен `f1_score`?

Удивительно, да?

Давайте разберемся, почему качество такое низкое.

Предскажем вероятности классов с помощью обученной логистической регрессии на тестовых данных.

In [20]:
probs_test = lr.predict_proba(Xtest)

probs_test[:10]

array([[0.86331972, 0.13668028],
       [0.86305957, 0.13694043],
       [0.80967732, 0.19032268],
       [0.73110167, 0.26889833],
       [0.87780414, 0.12219586],
       [0.96122864, 0.03877136],
       [0.95965491, 0.04034509],
       [0.96560659, 0.03439341],
       [0.92114871, 0.07885129],
       [0.9349701 , 0.0650299 ]])

По вероятностям видно, что вероятности отнесения к положительному классу очень низкие.

Попробуем изменить порог для перевода вероятности в классы.

**Вопрос**  
Чему равен `f1_score`, если все объекты с вероятностью не меньшей 0.1, относить к положительному классу?

Ответ округлите до сотых.

In [61]:
probs_test_1 = probs_test[:,1]
print(probs_test_1)
new_pred = np.array(list(map(lambda x: 1 if x >= 0.1 else 0, probs_test_1)))
print(new_pred)

print(round(f1_score(ytest, new_pred),2))

[0.13668028 0.13694043 0.19032268 ... 0.09132017 0.09727959 0.07518077]
[1 1 1 ... 0 0 0]
0.25


У обученной модели можно посмотреть веса (как и в линейной регрессии).  
Выведем на экран веса модели (`model.coef_`, `model.intercept_`).

In [62]:
lr.coef_, lr.intercept_

(array([[-5.55214705e-02, -7.90868782e-04, -4.28172545e-04,
         -1.07248154e-03, -1.34958177e-03, -4.95807875e-04,
          1.13131722e-05, -1.80011616e-03, -1.26843809e-03,
         -1.45283567e-05]]),
 array([-0.00115086]))

Создайте `pd.DataFrame`, где в первом столбце стоят названия признаков, а во втором - их веса (так удобнее анализировать результат).  
Отсортируйте таблицу по убыванию весов.

**Вопрос**  
Какой признак имеет наибольший положительный вес?

In [86]:
features = list(Xtest.columns.values) # нужны только листы, чтобы закидывать в датафрейм
print(features)
print(len(features))
coefs = list(lr.coef_[0,:]) # там список в списке ...
print(coefs)
print(len(coefs))
X_and_w = pd.DataFrame({'features': features, 'coefs': coefs})
X_and_w.sort_values('coefs', ascending=False, inplace=True)
print(X_and_w)

['AGE', 'SOCSTATUS_WORK_FL', 'SOCSTATUS_PENS_FL', 'GENDER', 'CHILD_TOTAL', 'DEPENDANTS', 'PERSONAL_INCOME', 'LOAN_NUM_TOTAL', 'LOAN_NUM_CLOSED', 'LOAN_DLQ_NUM']
10
[-0.05552147051932881, -0.0007908687815780321, -0.00042817254510100827, -0.0010724815421906185, -0.0013495817666406645, -0.0004958078745643626, 1.1313172190486572e-05, -0.0018001161554912857, -0.0012684380861596896, -1.4528356736766587e-05]
10
            features     coefs
6    PERSONAL_INCOME  0.000011
9       LOAN_DLQ_NUM -0.000015
2  SOCSTATUS_PENS_FL -0.000428
5         DEPENDANTS -0.000496
1  SOCSTATUS_WORK_FL -0.000791
3             GENDER -0.001072
8    LOAN_NUM_CLOSED -0.001268
4        CHILD_TOTAL -0.001350
7     LOAN_NUM_TOTAL -0.001800
0                AGE -0.055521


## Бонус

**Задание 1**  
Подберите порог для перевода вероятностей в классы, дающий максимальное значение `f1_score`.

In [87]:
thresholds = np.arange(0, 1.1, 0.1)
probabilities_test_1 = probs_test[:,1]
f1_scores = []
for threshold in thresholds:
    probs_test_1 = probs_test[:,1]
    new_pred = np.array(list(map(lambda x: 1 if x >= threshold else 0, probabilities_test_1)))
    f1 = f1_score(ytest, new_pred)
    f1_scores.append(f1)

best_threshold = thresholds[np.argmax(f1_scores)]
best_f1_score = max(f1_scores)

print("Лучший порог:", best_threshold)
print("Максимальное значение f1_score:", best_f1_score)


Лучший порог: 0.1
Максимальное значение f1_score: 0.2469521725539231


**Задание 2**  
Во вложенном цикле подберите одновременно коэффициент регуляризации `C` у логистической регрессии и порог для перевода вероятностей в классы, дающие максимальное значение `f1_score`.

По-хорошему, чтобы не переобучиться, эти величины надо подбирать не по тесту, а по отдельной выборке.

Поэтому разобъем данные изначально на три части: `Xtrain`, `Xval`, `Xtest`.

*   В цикле при подборе `С` и порога будем обучаться по `Xtrain`, а предсказывать и измерять качество по `Xval`.

*   Качество итоговой модели с найденными `C` и порогом измерьте по `Xtest`.

Так не переобучимся!


In [104]:
Xtrain_new, Xval, ytrain_new, yval = train_test_split(Xtrain, ytrain, train_size=0.7, random_state=123)

# ваш код для подбора C и порога здесь

best_f1_score = 0
best_C = None
best_threshold = None

# Перебор разных значений C и порогов на валидационном наборе
thresholds = np.arange(0, 1.1, 0.1)

for C in np.arange(0.1, 100.1, 0.5):  # Разные значения коэффициента регуляризации C
    model = LogisticRegression(C=C)
    model.fit(Xtrain_new, ytrain_new)
    probs_test = model.predict_proba(Xval)
    for threshold in thresholds:
        probs_test_1 = probs_test[:,1]
        predicted_labels = np.array(list(map(lambda x: 1 if x >= threshold else 0, probs_test_1)))
        f1 = f1_score(yval, predicted_labels)
        
        if f1 > best_f1_score:
            best_f1_score = f1
            best_C = C
            best_threshold = threshold


In [105]:
model = LogisticRegression(C = best_C)

model.fit(Xtrain, ytrain) # обучаемся на всех тренировочных данных

prediction = model.predict_proba(Xtest)
probs_test_1 = prediction[:,1]
predicted_labels = np.array(list(map(lambda x: 1 if x >= threshold else 0, probs_test_1)))
classes = predicted_labels # переведите полученные вероятности в классы по найденному порогу
print(classes)

[0 0 0 ... 0 0 0]


In [106]:
# вычислите значение f1_score на тестовых данных
f1_test = f1_score(ytest, classes)
print("Лучший порог:", best_threshold)
print("Лучшее значение C:", best_C)
print("Максимальное значение f1_score на валидационном наборе:", best_f1_score)
print("f1_score на тестовом наборе с лучшими параметрами:", f1_test)

Лучший порог: 0.1
Лучшее значение C: 0.1
Максимальное значение f1_score на валидационном наборе: 0.2270127584689837
f1_score на тестовом наборе с лучшими параметрами: 0.0
